## Preamble

Run these if notebook is in a new environment, otherwise just skip to `Import modules`

#### Boot cell

Skip if `module_update.py` is in same folder as notebook, next cell downloads this file which in turn will download other files.

In [1]:
!curl -s "https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py" > "module_update.py"

#### Get essentials

Import text and graph modules for analysis, and css for display

In [2]:
from module_update import update, css

In [3]:
css()

# Import modules

Aktiver cellen nedenfor for å få alt til å virke.

In [4]:
update('nbtext', overwrite=True)
import nbtext as nb
from IPython.display import HTML
import requests
from bs4 import BeautifulSoup


def mods_digibok(urn, kind='marcxml'):
    """find information about serial number urn (or full urn) for books using NB API. 
    Parameter 'kind' is one of mods, marcxml or dublincore """
    
    if not str(urn).startswith('URN'):
        urnstr = "URN:NBN:no-nb_digibok_{urn}".format(urn=urn)
    r = requests.get("https://api.nb.no:443/catalog/v1/metadata/{ident}/{kind}".format(ident=urnstr, kind=kind))
    return r.text

def translator(urn):
    """Finds the translator for a book"""
    item = BeautifulSoup(mods_digibok(urn, kind='mods'), 'lxml')
    trans = []
    navn = item.find_all('name', {'type':'personal'})
    for i in navn:

        try:
            if i.roleterm.text == 'trl':
                trans.append(i.namepart.text)
        except:
            'nn'
    return ';'.join(trans)
    

Updated file `C:\Users\larsj\Downloads\nbtext.py`

wordcloud er ikke installert, kan ikke lage ordskyer


## Definer korpuset 

Check out [Webdewey](http://deweysearchno.pansoft.de/webdeweysearch/index.html) for more descriptions.

Her har jeg satt `ddk` til *839%* som gir skjønnlitteratur. Har også satt trans = '', som gir originale norske verk.

In [18]:
korpus = nb.get_urn({
    'year':1990, 
    'next':20,     # fra 1990 til 2010
    'lang':'nob',
    'ddk': "839%", # Se bare på skjønnlitteratur
    'trans':'',    # Når trans settes til blank, '', vil verdien av lang også være originalspråk
    'limit':3000
})
len(korpus)

3000

# Hent korpus med størrelse

Her defineres korpuset som en liste over bøker, med informasjon om størrelse. 

In [19]:
rammen = nb.frame(nb.metadata(korpus), ['urn', 'forfatter','år','tittel','språk','forlag', 'original' ]).sort_values(by='tittel')
rammen = rammen.set_index('urn')
rammen.index = rammen.index.astype(str)


print('Oversettere klar - henter bokstørrelser')

book_counts = nb.frame(nb.book_count([x[0] for x in korpus]), ['antall_ord'])
book_counts.index = book_counts.index.astype(str)
book_counts.antall_ord = book_counts.antall_ord.astype(int)

print('Telling ferdig')

# sammenslåing

rammen = rammen.join(book_counts)
rammen = rammen[['forfatter',  'år', 'tittel', 'antall_ord', 'original']]
rammen.head()

Oversettere klar - henter bokstørrelser
Telling ferdig


,forfatter,år,tittel,antall_ord,original
urn,,,,,
2010072308057,"Ystad, Vigdis",1996,"""-livets endeløse gåde""",66599.0,
2011030109023,"Christensen, Lars Saabye",2000,"""Dracula""",27865.0,
2010092706047,"Berggren, Arne",1999,"""Du ser SÅ godt ut!""",21058.0,
2010072008005,"Angell-Jacobsen, Rune",1998,"""El Rubio""",71013.0,
2011061008006,"Paulsen, Irene",2004,"""En dag skal jeg komme hjem-""",16216.0,


## Se på hele korpuset under ett som HTML

Fyll in feltet bak `by` med et kolonnenavn for å sortere på verdiene i den kolonnen.

In [21]:
HTML(rammen.sort_values(by = 'original').to_html())

,forfatter,år,tittel,antall_ord,original
urn,,,,,
2010072308057,"Ystad, Vigdis",1996,"""-livets endeløse gåde""",66599.0,
2011090206079,"Imerslund, Knut",2005,Rau skulle kjolen vara,NaN,
2011031005102,"Rødsten, Sigurd",2000,Raver,6127.0,
2010051906035,"Hauger, Torill Thorstad",1992,Ravnejenta,70407.0,
2008102704170,"Gunnerud, Jørgen",2000,Raymond Isaksens utgang,70412.0,
2008101000041,"Nor-Hansen, Henrik",2000,Realitetene,6858.0,
2015071308029,"Ustad, Willy",2001,Rebecca,96129.0,
2011031520033,"Thon, Jahn",2001,Refleksjon - kritikk - protest,197771.0,
2009012000019,"Lindell, Unni",1998,Regjeringen bestemmer hvordan været skal bli,9315.0,


## Search corpus for concordances

In [8]:
from random import sample

In [9]:
nb.get_urnkonk('gråtende', {'urns':sample(korpus, 50)})

"Yankee pasja, Marshall, Edison, 1950",begrave dem . Kvalm og,gråtende,gjorde jeg det og skyndte
"Yankee pasja, Marshall, Edison, 1950",Hun fløy,gråtende,ut . Det beste jeg
"Gjennom India, Bourke-White, Margaret, 1951",Vi forlot den,gråtende,elefant og den sønneløse mahout
"Karavanen drar forbi, Tabori, George, 1952","Denef , » ropte han",gråtende,. « Denef ! »
"Lykkelige tider, Raverat, Gwen, 1952",ba om forlatelse og gikk,gråtende,"hjem , fullstendig knust av"
"Cherry på vingene, Wells, Helen, 1952","mor med et lite ,",gråtende,barn . Barnet hadde ikke
"På jakt etter den glemte by, Lamb, Dana, 1952",for å se « Den,gråtende,Madonna » . Til vår
"Arven, Croft-Cooke, Rupert, 1950",begravd ? » kom det,gråtende,fra Maud . « Men
"Muren, Hersey, John, 1951",seg ut øyeblikkelig. Rachel gikk,gråtende,sin vei .
"Muren, Hersey, John, 1951","til , Mauritzi Apt ,",gråtende,og med en nellik som
"Muren, Hersey, John, 1951",Hun reiste seg og løp,gråtende,inn i soveværelset . Berson
